In [ ]:
using Revise
using InstrumentControl
using InstrumentControl:E5071C, GS200, SMB100A, AWGM320XA, DigitizerM3102A 
using AxisArrays
using KeysightQubits
using KeysightInstruments
using JLD, FileIO, LsqFit
import Plots
Plots.plotlyjs()

import InstrumentControl: Stimulus, Response, Instrument, source, measure, PropertyStimulus

dB(x)= 20*log10(abs(x))
phas(x)=atan2(imag(x),real(x))

# Equipment Initialization

### Yokogawa GS200 DC Source

In [ ]:
yoko = InsGS200(tcpip_socket("169.254.65.200",7655))  #initializing instrument object
Vz = RampStim(yoko) 

### R&S SMB100A Signal Generators

In [ ]:
#iinitializing signal generator instrument objects
sigRead = InsSMB100A(tcpip_instr("169.254.1.20")) 
sigXY = InsSMB100A(tcpip_instr("169.254.235.224"))

In [ ]:
#defining stimuli for changing signal generator power and frequencies
xyPowerStim = PropertyStimulus(sigXY, PowerLevel, axisname=:xypower, axislabel="XY power (dBm)")
xyFreqStim = PropertyStimulus(sigXY, Frequency, axisname=:xyf, axislabel="XY frequency (GHz)");
readoutFreqStim = PropertyStimulus(sigRead, Frequency, axisname=:readf, axislabel="Readout frequency (GHz)");

### E5071C Vector Network Analyzer(VNA)

In [ ]:
vna = InsE5071C(tcpip_socket("169.254.61.208",5025)) #initializing signal generator instrument object
readavailable(vna)
#NOTE!: THE LAST COMMAND IS SUPPOSED TO GIVE AN VI_ERROR_TMO; THAT IS JUST A BUG

In [ ]:
S_ParamsResponse = VNA.FrequencySweep(vna) #redefinition for convenience; this is a response object which measures S parameters 
VNAPowerStim = PropertyStimulus(vna, PowerLevel, axisname=:vnapower, axislabel="VNA power (dBm)")

### Keysight Hardware

In [ ]:
#The inputs to the constructors are slot numbers
awg2 = InsAWGM320XA(2) 
awg4 = InsAWGM320XA(4)
dig = InsDigitizerM3102A(6)

In [ ]:
#Defining stimulus for DC biasing with slow AWG
key_Vz = Keysight_Vz(awg4, 1)  #inputs are: particular AWG in chassis, and channel number

# Finding Readout Resonator and Qubit Frequencies (Frequency Domain)

In [ ]:
#configuring VNA settings to measure S21 
vna[TriggerSource] = :InternalTrigger
vna[NumPoints] = 1001
vna[IFBandwidth] = 2000.
vna[Smoothing] = false
vna[Averaging] = true
vna[AveragingFactor] = 1
vna[VNA.Parameter] = :S21
vna[Timeout]=10000

In [ ]:
#imaging VNA S21 measurement at frequency range of interest --> Looking for resonator lorentzian
vna[PowerLevel] = -55
vna[FrequencyStart] = 6.883e9
vna[FrequencyStop] = 6.89e9
measure(S_ParamsResponse)
screen(vna)

In [ ]:
#Saturating resonator to see transition from |0> dispersed dressed resonance to bare cavity resonance
vna[FrequencyStart] = 6.875e9
vna[FrequencyStop] = 6.89e9

sweep(S_ParamsResponse, (VNAPowerStim, -60:5:-10))

In [ ]:
r = result(1)[:S21] #fill in job number here
Plots.heatmap(axisvalues(r)[2], axisvalues(r)[1]/1e9, dB.(r), xl="Readout power (dBm)", yl="VNA frequency (GHz)")

#### |0> dispersed frequency:
#### bare cavity frequency: 
#### χ = 

### Finding SQUID Frequency Period and Zero Flux Offset, in Volts 

In [ ]:
#Tuning qubit from max resonance to zero frequency with Z-line.If ω_r> ω_q_max, then as V_z is increased until it's... 
# half period value --> the qubit goes to zero frequency --> the detuning increases --> the dispersive shift decreases --> 
# readout resonance changes
# If ω_r < ω_q_max, then you will see an avoided level crossing 

vna[PowerLevel] = -60
vna[FrequencyStart] = 6.875e9
vna[FrequencyStop] = 6.89e9

sweep(S_ParamsResponse, (Vz, -0.8:0.025:0.8))

In [ ]:
r = result(2)[:S21]
Plots.heatmap(axisvalues(r)[2], axisvalues(r)[1]/1e9, dB.(r), xl="Vz (V)", yl="VNA frequency (GHz)")

#### V_offset = 
#### V_period = 

### Finding Qubit Frequency

In [ ]:
# measuring S21, in the vicinity of the readout resonator |0> dispersed frequency, while sweeping XY frequency with the... 
#... signal generator generator. If sigXY[Frequency] = frequency, the qubit is Rabi oscillating so fast compared to Ω that... 
#... the resonator only sees an average of the two qubit states, i.e. motional averaging
vna[PowerLevel] = -60
vna[FrequencyStart] = 6.880e9
vna[FrequencyStop] = 6.888e9
source(V_z, 0.025) #source voltage offset so qubit is at maximum frequency

sweep(S_ParamsResponse, (xyFreqStim, 4.2e9:1e6:5.2e9), (xyPowerStim, [-20, -30, -40, -50]))

In [ ]:
# -20dBm power
r = result(6058)[Axis{:sparam}(:S21), Axis{:xypower}(1)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
#-30dBm Power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(2)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
#-40dBm Power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(3)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
# -50dBm power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(4)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

# Finding Resonator/Qubit Frequencies and Rabi Period (Time Domain)

#### The delay (in number of digitizer samples) needed to start acquisition of I and Q signal after marker arrival at the digitizer was determined to be 125 samples = 250ns. Briefly, 20dB of attenuation after readout up-conversion, without using the on-rack 17dB amplifier (before down-conversion), was enough to obtain a 40mVpp pulse with IF amplitude = 1. This signal was then measured with the digitizer (i.e., the voltage levels), and the delay was determined to be 125 samples (250ns). The normal rack configuration (needed to reach low photon levels) uses 40dB of attenuation after up-conversion, along with a ~17dB amplifier prior to downconversion. The normal configuration leads to really noisy output (you can't make out a pulse by visual inspection of the signal), but with the demodulation to DC plus averaging, you can really pick out the IF tone

#### NOTE: We put ~ 30dB of attenuation on the XY line. Through testing, we saw that, even in the abscence of a pulse, we saw the frequency response of the cavity change in comparison to when sigXY is off AT VARIOUS FREQUENCIES. This implies carrier bleedthrough and the spectrum analyzer confirms that. To ameliorate the issue, we added attenuation on the output of the XY line and increased the amplitude voltage out of the AWG (for consistent XY power when outputting a pulse). This allows us to use high LO power for proper functioning of the IQ mixer, while still letting us bring down the carrier bleedthrough to manageable levels. Increasing the IF also further isolates the qubit from carrier bleedthrough.

#### Pulses, Stimulus, and Response Initialization

In [ ]:
πpulse_length = 400e-9
π_2pulse_length = 200e-9 #has to be a multiple of 10ns
xyIF = 175e6
readoutIF = 100e6
XY_amplitude = 0.25
readout_amplitude = 0.15

rabiPulse = AnalogPulse(xyIF, XY_amplitude, 0) #placeholder pulse, has no duration or waveforms. 0 corresponds to IF phase
Xπ = AnalogPulse(xyIF, XY_amplitude, πpulse_length, CosEnvelope, awg4[SampleRate])
Xπ_2 = AnalogPulse(xyIF, XY_amplitude, π_2pulse_length, CosEnvelope, awg4[SampleRate])
readout = DigitalPulse(readoutIF, readout_amplitude, 500e-9, RectEnvelope, awg4[SampleRate])

#the functions take for standard arguments: marker channel = 4, control PXI line = 0, and standard axisname and axislabel inputs 
stim_readRef = ReadoutReference(awg4, awg2, readout, 60e-6, (2,3))
stim_Rabi = Rabi(awg4, awg4, awg2, rabiPulse, readout, 60e-6, 60e-6, (1,4), (2,3))
stim_T1 = T1(awg4, awg4, awg2, Xπ, readout, 60e-6, 60e-6, (1,4), (2,3))
stim_Ramsey = Ramsey(awg4, awg4, awg2, Xπ_2, readout, 60e-6, 60e-6, (1,4), (2,3))

num_averages = 5000
readout_num_samples = readout.duration*dig[SampleRate]
resp_IQ = IQTrigResponse(dig, 1, 2, num_averages, readout_num_samples, 125, :TRGPort, 100e6) #averaging for 10000 times
_avgIQnew = Avg_IQResponse(resp_IQ);

### Finding Resonator Frequency

In [ ]:
source(Vz, 0.025) #source voltage offset so qubit is at maximum frequency
configure_awgs(stim_readRef)
source(stim_readRef)
sweep(_avgIQnew, (readoutFreqStim, (6.980e9:0.1e6:6.989e9)))

In [ ]:
r = result(6257)
Plots.plot(axisvalues(r)[1]/1e9, abs.(r), xl="Readout LO Frequency (GHz)", yl="IF Amplitude")

#### |0> dispersed resonator frequency:

### Finding Qubit Frequency 

In [ ]:
source(readoutFreqStim, 6.9867e9) #this should be the resonator's dispersed |0> frequency
source(Vz, 0.032) #source voltage offset so qubit is at maximum frequency
configure_awgs(stim_Rabi)
source(stim_Rabi, πpulse_length) 
sweep(_avgIQnew, (xyFreqStim, 4.65e9:0.5e6:4.9e9))

In [ ]:
r = result(6292)
Plots.plot(axisvalues(r)[1]/1e9, abs.(r), xl="XY LO Frequency (GHz)", yl="IF Amplitude")

#### Qubit frequency:

### Finding Rabi Period

In [ ]:
source(readoutFreqStim, 6.9867e9) #this should be the resonator's dispersed |0> frequency
source(xyFreqStim, 4.87e9)
source(Vz, 0.025) #source voltage offset so qubit is at maximum frequency
sweep(_avgIQnew, (stim_Rabi, 20e-9:10e-9:2e-6))

In [ ]:
r = result(6293)
Plots.plot(axisvalues(r)[1]*1e9, abs.(r), xl="Time (ns)", yl="IF Amplitude" )

#### Pi pulse length: 

# Chevron, T1, and T2*

### Chevron Pattern

In [ ]:
source(readoutFreqStim, 6.9867e9) #this should be the resonator's dispersed |0> frequency
source(Vz, 0.025) #source voltage offset so qubit is at maximum frequency
sweep(_avgIQnew, (stim_Rabi, 20e-9:10e-9:2e-6), (xyFreqStim, 4.86e9:1e6:4.88e9))

In [ ]:
r = result(6239)
Plots.heatmap(axisvalues(r)[1]*1e9, (axisvalues(r)[2] - xyIF)/1e9, dB.(r), xl="Pulse duration (ns)", yl="XY Frequency (GHz)")

### Measuring T1

In [ ]:
source(readoutFreqStim, 6.9867e9) #this should be the resonator's dispersed |0> frequency
source(xyFreqStim, 4.87e9)
source(Vz, 0.025) #source voltage offset so qubit is at maximum frequency
configure_awgs(stim_T1)
sweep(_avgIQnew, (stim_T1, 20e-9:100e-9:50e-6))

In [ ]:
r = result(6308)
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (μs)" , yl = "Amplitude", label = "Data")

In [ ]:
Plots.plot(axisvalues(r)[2]*1e6, abs.(r), xl = "Delay (μs)", yl = "IF Amplitude", label = "Data")
t1fit(x,p) = @. p[1]*exp(-x/p[2])+p[3]
fit = curve_fit(t1fit, axisvalues(r)[2]*1e6, Float64.(abs.(r)), [0.0029, 20, 1])
Plots.plot!(axisvalues(r)[2]*1e6, t1fit(axisvalues(r)[2]*1e6, fit.param), label = "Fit T1 ="*string(fit.param[2])[1:4]*"μs")

### Measuring T2*

In [ ]:
source(readoutFreqStim, 6.9867e9) #this should be the resonator's dispersed |0> frequency
source(xyFreqStim, 4.87e9)
source(Vz, 0.025) #source voltage offset so qubit is at maximum frequency
configure_awgs(stim_Ramsey)
sweep(_avgIQnew, (stim_Ramsey, 20e-9:100e-9:50e-6))

In [ ]:
r = result(6308)
Plots.plot(axisvalues(r)[1]*1e6, abs.(r), xl = "Delay (μs)" , yl = "Amplitude", label = "Data")

In [ ]:
t2fit(x,p) = @. p[1]*exp(-x/p[2])*cos(x*(2π/p[3])-p[4])+p[5]
fit = curve_fit(t2fit, axisvalues(r)[2]*1e6, Float64.(abs.(r)), [1, 1 ,1, 1, 0.00286358])
Plots.plot(axisvalues(r)[2]*1e6, abs.(r), xl = "Delay (ns)" , yl = "Amplitude", label = "Data")
Plots.plot!(axisvalues(r)[2]*1e6, t2fit(axisvalues(r)[2]*1e6, fit.param), label = "Fit T2 ="*string(fit.param[2])[1:4]*"μs")